# Veriler'in Özetlenmesi ve Temizlenmesi Algoritması

-Bu kod'un içeriği ve çalışması ortalama 800.000 veri için 6 saat sürüyor, bilginize...

- Kod'un içeriği ise öncelikle, MüşteriID'leri çekiliyor, Ardından 10 İşlem üstü müşteriler alınıyor. Ardından Son işlem üzerinden 6 ay geçmiş mi o kontrol ediliyor. Eğer geçtiyse Churn geçmediyse churn değil şeklinde işaretleniyor. Ve kalan 19.990 Müşteri için Teker teker işlemler tek satıra özetleniyor, Özetleme olarak Kategorik değişkenler için frekanslarına bakılıyor, Sürekliler içinse Son 5 İşlem'in ortalaması alınıyor.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import itemfreq
from string import ascii_letters
import seaborn as sns
import matplotlib.pyplot as plt
from progressbar import *

In [2]:
data = pd.read_excel("All-in-one.xlsx")
data.head()

,ID,CEK_NO,CEK_TUTAR,VADE_TARIH,VADE_GUN,ISTIHBARAT_SONUC,POLITIKA_SONUC,CEK_RENK,KULLANDIRIM,SUBE,...,T4_NAKDIRISK_TPL,T4_NAKDIRISK_KV,T4_NAKDIRISK_OV,T4_NAKDIRISK_UV,T4_GAYRINAKDILIMIT,T4_GAYRINAKDIRISK,T4_FKTRNG_LIMIT,T4_FKTRNG_TPLRISK,T4_SORUNLUKOD_RISK,Unnamed: 63
0,10392519,65450,20000.0,2017-06-30,179,OTO-RED,HAYR,Siyah,KY,MECIDIYEKÖY,...,4150.0,100.0,0.0,4050.0,77000.0,20640.0,0.0,0.0,0.0,NaN
1,10392520,4973613,42000.0,2017-03-02,59,OTO-RED,HAYR,Siyah,KY,OSTIM,...,0.0,0.0,0.0,0.0,42210.0,28020.0,350000.0,17952.0,0.0,NaN
2,10392521,9515891,30000.0,2017-06-25,174,OTO-RED,HAYR,Sari,KY,OSTIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10392522,9515890,30000.0,2017-05-25,143,OTO-RED,HAYR,Sari,KY,OSTIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10392523,9515889,20000.0,2017-04-20,108,OTO-RED,HAYR,Yesil,KY,OSTIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- musteriidcekici fonksiyonu sizin belirlediğiniz sınır'ın üstünde işlemi olan kişileri index listesinde bazında alıyor.
- indexer fonksiyonu ise aldığınız index listesini gerçek data içerisinde filtreliyor.
- churnlabelling fonksiyonu ise elimizdeki verinin son tarihine göre alıyor (Örnek vermek gerekirse elimizdeki verideki son tarih 2018 kasım, bunun 6 ay öncesi alınıyor, 2018 Haziran ve buna göre ilk işlemi 2018 hazirandan sonra olanlar veriden çıkartılıyor, ve kişinin son işlemi üzerinden 6 ay geçtiyse bu kişi churn olarak etiketleniyor, geçmediyse bu kişi normal olarak etiketleniyor.

- **Eğer bu kısımı anlamadıysanız sunumdaki 5 sayfadaki slayta bakmanız yeterli olacaktır.** 

In [4]:
def musteriidcekici(data,sınır):
    musteridata = pd.DataFrame(data.groupby('MUSTERI_ID').size())
    musteridata = musteridata[musteridata[0] > sınır]
    musteriindexlist = list(musteridata.index)
    return musteriindexlist

def indexer(churnname,data):
    dataindex = data.set_index("MUSTERI_ID")
    churnname = np.array(churnname)
    churndata = dataindex.loc[churn10]
    return churndata

def churnlabelling(dataname,churnname):
    global nonchurn,churn
    nonchurn = []
    churn = []
    d = "Jun 01 2018"
    c = 0
    date = datetime.datetime.strptime(d, '%b %d %Y')
    for i in churnname:
        tarih = dataname.loc[i].iloc[len(dataname.loc[i])-1]["ISLEM_TARIHI"].strftime('%Y-%m-%d')
        tarih = datetime.datetime.strptime(tarih, '%Y-%m-%d')
        tarih1 = dataname.loc[i].iloc[0]["ISLEM_TARIHI"].strftime('%Y-%m-%d')
        tarih1 = datetime.datetime.strptime(tarih1, '%Y-%m-%d')
        if tarih1 > date:
            continue
        else:
            if (tarih - date).days < 180:
                nonchurn.append(i)
            elif (tarih - date).days > 180:
                churn.append(i)

In [5]:
churn10 = musteriidcekici(data,10)

In [6]:
churndata = indexer(churn10,data)

In [7]:
churnlabelling(churndata,churn10)

In [8]:
# Little İnfo About v1
churn10 = musteriidcekici(data,10)
churn9 = musteriidcekici(data,9)
churn8 = musteriidcekici(data,8)
churn7 = musteriidcekici(data,7)
churn6 = musteriidcekici(data,6)
churn5 = musteriidcekici(data,5)

print("En az 5 İşlem İçin Müşteri Sayımız : ",len(churn5))
print("En az 6 İşlem İçin Müşteri Sayımız : ",len(churn6))
print("En az 7 İşlem İçin Müşteri Sayımız : ",len(churn7))
print("En az 8 İşlem İçin Müşteri Sayımız : ",len(churn8))
print("En az 9 İşlem İçin Müşteri Sayımız : ",len(churn9))
print("En az 10 İşlem İçin Müşteri Sayımız : ",len(churn10))

En az 5 İşlem İçin Müşteri Sayımız :  34933
En az 6 İşlem İçin Müşteri Sayımız :  31160
En az 7 İşlem İçin Müşteri Sayımız :  28065
En az 8 İşlem İçin Müşteri Sayımız :  25604
En az 9 İşlem İçin Müşteri Sayımız :  23515
En az 10 İşlem İçin Müşteri Sayımız :  21607


In [9]:
len(churn) , len(nonchurn)

(1615, 18375)

In [10]:
churnsdata = churndata.loc[churn]
nonchurndata = churndata.loc[nonchurn]
churnsdata["Churn"] = 1
nonchurndata["Churn"] = 0

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
labelleddata = pd.concat([churnsdata,nonchurndata], axis=0)

# Temizlenme Aşaması

- Bu kısımda ise Tarih, ID gibi unique değişkenleri veriden çıkartıyorum, ve istihbaharat için rahatça ortalamasını alabilmek için herşeyi ONAY/RED/İPTAL şeklinde etiketliyorum.

In [12]:
labelleddata.drop(["Unnamed: 63","ID","CEK_NO","VADE_TARIH","KESIDECI_ID","ISLEM_NO","KURULUS_TARIH","TK_ILKKREDIKULLANDIRIMTAR","TK_SONKREDIKULLANDIRIMTAR","BK_SONKREDIKULLANDIRIMTAR"],axis=1,inplace=True)
labelleddata

,CEK_TUTAR,VADE_GUN,ISTIHBARAT_SONUC,POLITIKA_SONUC,CEK_RENK,KULLANDIRIM,SUBE,ISLEM_TARIHI,MUSTERI_RISK_SEVIYESI,SIRKET_TURU,...,T4_NAKDIRISK_TPL,T4_NAKDIRISK_KV,T4_NAKDIRISK_OV,T4_NAKDIRISK_UV,T4_GAYRINAKDILIMIT,T4_GAYRINAKDIRISK,T4_FKTRNG_LIMIT,T4_FKTRNG_TPLRISK,T4_SORUNLUKOD_RISK,Churn
MUSTERI_ID,,,,,,,,,,,,,,,,,,,,,
480,8567.00,22,OTO-ONAY,EVET,Yesil,KY,IKITELLI,2017-05-05,0,G,...,2553.0,2553.0,0.0,0.0,815.0,815.0,50000.0,0.0,0.0,1
480,15000.00,80,OTO-ONAY,EVET,Sari,KY,IKITELLI,2017-05-12,0,G,...,2553.0,2553.0,0.0,0.0,815.0,815.0,50000.0,0.0,0.0,1
480,9771.58,100,OTO-ONAY,EVET,Turuncu,KV,IKITELLI,2017-12-15,0,G,...,1346.0,1346.0,0.0,0.0,815.0,815.0,57950.0,7950.0,0.0,1
480,18000.00,184,ONAY,EVET,Sari,KV,IKITELLI,2018-05-18,0,G,...,748.0,748.0,0.0,0.0,940.0,940.0,190000.0,9772.0,0.0,1
480,18066.00,163,OTO-ONAY,EVET,Yesil,KV,IKITELLI,2018-06-20,0,G,...,498.0,498.0,0.0,0.0,940.0,940.0,190000.0,9772.0,0.0,1
480,23400.00,65,OTO-ONAY,EVET,Yesil,KV,IKITELLI,2018-07-27,0,G,...,1155.0,1155.0,0.0,0.0,940.0,940.0,190000.0,0.0,0.0,1
480,20000.00,100,OTO-ONAY,EVET,Sari,KV,IKITELLI,2018-08-17,0,G,...,960.0,960.0,0.0,0.0,940.0,940.0,190000.0,0.0,0.0,1
480,18410.00,87,OTO-RED,HAYR,Siyah,KY,IKITELLI,2018-10-12,0,G,...,1752.0,940.0,0.0,812.0,0.0,0.0,190000.0,35782.0,0.0,1
480,18410.00,84,RED,EVET,Sari,KY,IKITELLI,2018-10-15,0,G,...,1752.0,940.0,0.0,812.0,0.0,0.0,190000.0,35782.0,0.0,1


In [13]:
istihbaharatmapping = {"OTO-ONAY":"ONAY","OTO-RED":"RED","IPTAL":"IPTAL","IADE":"IPTAL","SF":"RED","ONAY":"ONAY","RED":"RED"}
labelleddata.ISTIHBARAT_SONUC = labelleddata.ISTIHBARAT_SONUC.map(istihbaharatmapping)
labelleddata

,CEK_TUTAR,VADE_GUN,ISTIHBARAT_SONUC,POLITIKA_SONUC,CEK_RENK,KULLANDIRIM,SUBE,ISLEM_TARIHI,MUSTERI_RISK_SEVIYESI,SIRKET_TURU,...,T4_NAKDIRISK_TPL,T4_NAKDIRISK_KV,T4_NAKDIRISK_OV,T4_NAKDIRISK_UV,T4_GAYRINAKDILIMIT,T4_GAYRINAKDIRISK,T4_FKTRNG_LIMIT,T4_FKTRNG_TPLRISK,T4_SORUNLUKOD_RISK,Churn
MUSTERI_ID,,,,,,,,,,,,,,,,,,,,,
480,8567.00,22,ONAY,EVET,Yesil,KY,IKITELLI,2017-05-05,0,G,...,2553.0,2553.0,0.0,0.0,815.0,815.0,50000.0,0.0,0.0,1
480,15000.00,80,ONAY,EVET,Sari,KY,IKITELLI,2017-05-12,0,G,...,2553.0,2553.0,0.0,0.0,815.0,815.0,50000.0,0.0,0.0,1
480,9771.58,100,ONAY,EVET,Turuncu,KV,IKITELLI,2017-12-15,0,G,...,1346.0,1346.0,0.0,0.0,815.0,815.0,57950.0,7950.0,0.0,1
480,18000.00,184,ONAY,EVET,Sari,KV,IKITELLI,2018-05-18,0,G,...,748.0,748.0,0.0,0.0,940.0,940.0,190000.0,9772.0,0.0,1
480,18066.00,163,ONAY,EVET,Yesil,KV,IKITELLI,2018-06-20,0,G,...,498.0,498.0,0.0,0.0,940.0,940.0,190000.0,9772.0,0.0,1
480,23400.00,65,ONAY,EVET,Yesil,KV,IKITELLI,2018-07-27,0,G,...,1155.0,1155.0,0.0,0.0,940.0,940.0,190000.0,0.0,0.0,1
480,20000.00,100,ONAY,EVET,Sari,KV,IKITELLI,2018-08-17,0,G,...,960.0,960.0,0.0,0.0,940.0,940.0,190000.0,0.0,0.0,1
480,18410.00,87,RED,HAYR,Siyah,KY,IKITELLI,2018-10-12,0,G,...,1752.0,940.0,0.0,812.0,0.0,0.0,190000.0,35782.0,0.0,1
480,18410.00,84,RED,EVET,Sari,KY,IKITELLI,2018-10-15,0,G,...,1752.0,940.0,0.0,812.0,0.0,0.0,190000.0,35782.0,0.0,1


In [14]:
#indexlist = list(labelleddata.index)

#newdataindex = []
#for i in indexlist:
#    if i not in newdataindex:
#        newdataindex.append(i)

- Bu kısımda indexlerle alakalı bir sorun oluyordu, indexteki aynı olan değerleri düşürüyordum, bu işlem çok uzun sürdüğü için dosyayı direkt pickle'dan çekmenin daha mantıklı olduğunun farkına vardım.

In [22]:
PIK = "indexlist.dat"
with open(PIK, "rb") as f:
    newdataindex =  pickle.load(f)

# Veriyi Özetlemek
- Bu kısımda tanıttığım dosya yukarıdaki işlemlerin dosyayı, birdaha birdaha çalıştırmamak için kaydedip okutmak daha mantıklı diye düşündüm. 

- Burada birkaç fonksiyonumuz var ilk olarak :
- **nanfiller** Bu fonksiyon risk raporundaki boş değerleri 0 olarak işaretliyor bunun sebebi ise **sunum'un 8. slaytında yer alıyor.**
- **createnewdata** , **createnewdatav2**  ,**createnewdatav3** bu 3 fonksiyon'un yaptığı iş aynı ama hitap ettiği kesim farklı, 3 fonksiyonda verinin tipine bakarak, (Kategorik, Sürekli) Değişkene göre o müşteri için **SON5** işlem baz alınarak ortalamasını alıyor, yada **Son5** işlemin frekanslarına bakarak en yüksek frekanslı değeri seçiyor. Buradaki Son5 yaklaşımı da biraz analiz yaptığımızda müşteriler için önce son işlemi baz almayı düşündük, ardından elimizde veri varken son5 işleme bakalım anlamlı sonuçlar çıkacak mı diye kontrol ettik. Ve anlamlı sonuçlar elde ettiğimiz için son5 işlemi baz almayı tercih ettik.

- Not : Burada diğerlerinden farklı olarak VADE_GUN VE CEK_TUTAR değişkenlerinin ortalamaları alınırken son5 işlem değil bütün işlemler baz alındı.

In [23]:
labelleddata = pd.read_excel("labelData.xlsx")
labelleddata

,MUSTERI_ID,CEK_TUTAR,VADE_GUN,ISTIHBARAT_SONUC,POLITIKA_SONUC,CEK_RENK,KULLANDIRIM,SUBE,ISLEM_TARIHI,MUSTERI_RISK_SEVIYESI,...,T4_NAKDIRISK_TPL,T4_NAKDIRISK_KV,T4_NAKDIRISK_OV,T4_NAKDIRISK_UV,T4_GAYRINAKDILIMIT,T4_GAYRINAKDIRISK,T4_FKTRNG_LIMIT,T4_FKTRNG_TPLRISK,T4_SORUNLUKOD_RISK,Churn
0,480,8567.00,22,ONAY,EVET,Yesil,KY,IKITELLI,2017-05-05,0,...,2553.0,2553.0,0.0,0.0,815.0,815.0,50000.0,0.0,0.0,1
1,480,15000.00,80,ONAY,EVET,Sari,KY,IKITELLI,2017-05-12,0,...,2553.0,2553.0,0.0,0.0,815.0,815.0,50000.0,0.0,0.0,1
2,480,9771.58,100,ONAY,EVET,Turuncu,KV,IKITELLI,2017-12-15,0,...,1346.0,1346.0,0.0,0.0,815.0,815.0,57950.0,7950.0,0.0,1
3,480,18000.00,184,ONAY,EVET,Sari,KV,IKITELLI,2018-05-18,0,...,748.0,748.0,0.0,0.0,940.0,940.0,190000.0,9772.0,0.0,1
4,480,18066.00,163,ONAY,EVET,Yesil,KV,IKITELLI,2018-06-20,0,...,498.0,498.0,0.0,0.0,940.0,940.0,190000.0,9772.0,0.0,1
5,480,23400.00,65,ONAY,EVET,Yesil,KV,IKITELLI,2018-07-27,0,...,1155.0,1155.0,0.0,0.0,940.0,940.0,190000.0,0.0,0.0,1
6,480,20000.00,100,ONAY,EVET,Sari,KV,IKITELLI,2018-08-17,0,...,960.0,960.0,0.0,0.0,940.0,940.0,190000.0,0.0,0.0,1
7,480,18410.00,87,RED,HAYR,Siyah,KY,IKITELLI,2018-10-12,0,...,1752.0,940.0,0.0,812.0,0.0,0.0,190000.0,35782.0,0.0,1
8,480,18410.00,84,RED,EVET,Sari,KY,IKITELLI,2018-10-15,0,...,1752.0,940.0,0.0,812.0,0.0,0.0,190000.0,35782.0,0.0,1
9,480,18410.00,61,RED,HAYR,Siyah,KY,IKITELLI,2018-11-07,0,...,3251.0,2451.0,0.0,800.0,0.0,0.0,190000.0,59093.0,0.0,1


In [24]:
PIK = "indexlist.dat"
with open(PIK, "rb") as f:
    newdataindex =  pickle.load(f)

In [25]:
columnsslist = ['T2_NAKDILIMIT_TPL', 'T2_NAKDIRISK_TPL', 'T2_NAKDIRISK_KV',
       'T2_GAYRINAKDIRISK',"T2_GAYRINAKDILIMIT", 'T2_FKTRNG_LIMIT', 'T2_FKTRNG_TPLRISK',
       'T2_NAKDIRISK_OV', 'T2_NAKDIRISK_UV', 'T2_SORUNLUKOD_RISK',
       'T3_NAKDILIMIT_TPL', 'T3_NAKDIRISK_TPL', 'T3_NAKDIRISK_KV',
       'T3_NAKDIRISK_OV', 'T3_NAKDIRISK_UV', 'T3_GAYRINAKDIRISK',
        "T3_GAYRINAKDILIMIT","T4_GAYRINAKDILIMIT",
       'T3_FKTRNG_LIMIT', 'T3_SORUNLUKOD_RISK', 'T3_FKTRNG_TPLRISK',
       'T4_NAKDILIMIT_TPL', 'T4_NAKDIRISK_TPL', 'T4_NAKDIRISK_KV',
       'T4_NAKDIRISK_OV', 'T4_NAKDIRISK_UV', 'T4_GAYRINAKDIRISK',
       'T4_FKTRNG_LIMIT', 'T4_SORUNLUKOD_RISK', 'T4_FKTRNG_TPLRISK']

def nanfiller(columnsslist):
    for i in columnsslist:
        labelleddata[i] = labelleddata[i].fillna(0)

In [26]:
nanfiller(columnsslist)

In [27]:
widgets = ['Loading: ', Percentage(), ' ', Bar(marker='#',left='[',right=']'),
           ' '] #see docs for other options
pbar = ProgressBar(widgets=widgets, maxval=len(newdataindex))
pbar.start()

In [28]:
def createnewdata(newdata,labelleddata):
    p = 0
    for i in newdataindex:
        newdata.VADE_GUN[i] = labelleddata.loc[i]["VADE_GUN"].mean()
        newdata.CEK_TUTAR[i] = labelleddata.loc[i]["CEK_TUTAR"].mean()
        newdata.ŞUBE[i] = labelleddata.loc[i]["SUBE"].values[1]
        ck = len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["ISTIHBARAT_SONUC"].values))
        newdata.SON5ISTIHBARAT_SONUC[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["ISTIHBARAT_SONUC"].values)[ck-1][0]
        cl = len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["POLITIKA_SONUC"].values))
        newdata.SON5POLITIKA_SONUC[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["POLITIKA_SONUC"].values)[cl-1][0]
        cm = len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["CEK_RENK"].values))
        newdata.SON5CEK_RENK_ORTALAMASI[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["CEK_RENK"].values)[cm-1][0]
        cp = len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["KULLANDIRIM"].values))
        newdata.SON5KULLANDIRIM[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["KULLANDIRIM"].values)[cp-1][0]
        
def createnewdatav2(newdata,labelleddata):
    c = 0
    for i in newdataindex:
        c = c + 1
        pbar.update(c)
        newdata.SIRKET_TURU[i] = labelleddata.loc[i]["SIRKET_TURU"].values[1]
        newdata.MUSTERI_RISK_SEVIYESI[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["MUSTERI_RISK_SEVIYESI"].values)[len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["MUSTERI_RISK_SEVIYESI"].values))-1][0]
        newdata.TK_KURUMSAYISISON5[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_KURUMSAYISI"].values)[len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_KURUMSAYISI"].values))-1][0]
        newdata.TK_GECIKMEHESAPSON5[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_GECIKMEHESAP"].values)[len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_GECIKMEHESAP"].values))-1][0]
        newdata.TK_GECIKMEBAKIYEORT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_GECIKMEBAKIYE"].mean()
        newdata.TK_NAKDILIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_NAKDILIMIT"].mean()
        newdata.TK_NAKDIRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_NAKDIRISK"].mean()
        newdata.TK_GAYRINAKDILIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["TK_GAYRINAKDILIMIT"].mean()
        newdata.BK_KURUMSAYISI[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_KURUMSAYISI"].values)[len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_KURUMSAYISI"].values))-1][0]
        newdata.BK_LIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_LIMIT"].mean()
        newdata.BK_RISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_RISK"].mean()
        newdata.BK_GECIKMEHESAP[i] = itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_GECIKMEHESAP"].values)[len(itemfreq(labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_GECIKMEHESAP"].values))-1][0]
        newdata.BK_GECIKMEBAKIYE[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["BK_GECIKMEBAKIYE"].mean()
    pbar.finish()

def createnewdatav3(newdata,labelleddata):
    k = 0
    for i in newdataindex:
        k = k + 1
        pbar.update(k)
        newdata.SIRKET_TURU[i] = labelleddata.loc[i]["SIRKET_TURU"].values[1]
        newdata.T2_NAKDILIMIT_TPL[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_NAKDILIMIT_TPL"].mean()
        newdata.T2_NAKDIRISK_TPL[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_NAKDIRISK_TPL"].mean()
        newdata.T2_NAKDIRISK_KV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_NAKDIRISK_KV"].mean()
        newdata.T2_NAKDIRISK_OV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_NAKDIRISK_OV"].mean()
        newdata.T2_NAKDIRISK_UV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_NAKDIRISK_UV"].mean()
        newdata.T2_GAYRINAKDILIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_GAYRINAKDILIMIT"].mean()
        newdata.T2_GAYRINAKDIRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_GAYRINAKDIRISK"].mean()
        newdata.T2_FKTRNG_LIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_FKTRNG_LIMIT"].mean()
        newdata.T2_FKTRNG_TPLRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_FKTRNG_TPLRISK"].mean()
        newdata.T2_SORUNLUKOD_RISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T2_SORUNLUKOD_RISK"].mean()
        newdata.T3_NAKDILIMIT_TPL[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_NAKDILIMIT_TPL"].mean()
        newdata.T3_NAKDIRISK_TPL[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_NAKDIRISK_TPL"].mean()
        newdata.T3_NAKDIRISK_KV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_NAKDIRISK_KV"].mean()
        newdata.T3_NAKDIRISK_OV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_NAKDIRISK_OV"].mean()
        newdata.T3_NAKDIRISK_UV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_NAKDIRISK_UV"].mean()
        newdata.T3_GAYRINAKDILIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_GAYRINAKDILIMIT"].mean()
        newdata.T3_GAYRINAKDIRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_GAYRINAKDIRISK"].mean()
        newdata.T3_FKTRNG_LIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_FKTRNG_LIMIT"].mean()
        newdata.T3_FKTRNG_TPLRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_FKTRNG_TPLRISK"].mean()
        newdata.T3_SORUNLUKOD_RISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T3_SORUNLUKOD_RISK"].mean()
        newdata.T4_NAKDILIMIT_TPL[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_NAKDILIMIT_TPL"].mean()
        newdata.T4_NAKDIRISK_TPL[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_NAKDIRISK_TPL"].mean()
        newdata.T4_NAKDIRISK_KV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_NAKDIRISK_KV"].mean()
        newdata.T4_NAKDIRISK_OV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_NAKDIRISK_OV"].mean()
        newdata.T4_NAKDIRISK_UV[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_NAKDIRISK_UV"].mean()
        newdata.T4_GAYRINAKDILIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_GAYRINAKDILIMIT"].mean()
        newdata.T4_GAYRINAKDIRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_GAYRINAKDIRISK"].mean()
        newdata.T4_FKTRNG_LIMIT[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_FKTRNG_LIMIT"].mean()
        newdata.T4_FKTRNG_TPLRISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_FKTRNG_TPLRISK"].mean()
        newdata.T4_SORUNLUKOD_RISK[i] = labelleddata.loc[i][len(labelleddata.loc[i]) - 5:]["T4_SORUNLUKOD_RISK"].mean()
        newdata.Churn[i] = labelleddata.loc[i]["Churn"].values[1]
    pbar.finish()

In [ ]:
#createnewdatav3(newdata,labelleddata)

In [ ]:
#newdata.to_excel("newDataFullv3.xlsx")